This is my Fantasy Hockey Analyzer. The purpose of this project is to predict the fantasy hockey output of individual skaters based on stats from previous years.


Section 1: Modules Used

The following is a list of modules that I used and the reason why they were used:

-os: to allow the program to read data in the repository

-numpy: basic math operations

-pandas: all dataframe operations/data storage/data cleaning

-various sklearn: all machine learning operations/analysis

In addition to these modules, I also have a custom module that contains helper functions that help in data cleaning/accuracy evaluation. These functions are contained in the "my_module.py" file in the repository. If you are interested in taking a look at these functions, they are available at https://github.com/chrisberry888/FantasyHockeyAnalyzer in the "my_module.py" file.

In [1]:
#Import block
import os
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import my_module as mx

Section 2: Data Gathering and Cleaning

The ultimate goal of this project is to predict the number of fantasy points that a given player will have in the 2022-2023 season. Different fantasy leagues have different points breakdowns, but my current league has a points breakdown as described in the "points_dict" variable in the following code block. For example, each player gets 5 fantasy points for a goal, 3 for an assist, and so on. Of course, this can be changed if a different league has a different points breakdown.

To accomplish this all, I have gathered data from rotowire.com and moneypuck.com. The Moneypuck data contains just about every advanced stat you could think of in a variety of different situations (5-on-5, 5-on-4, etc). However, for the sake of this project we will only use their data that describes a player's output in all of their situations. The only stat we need that can't be calculated using the Moneypuck data is +/-; Rotowire has +/- available, so I'm using that dataset.

Data from these sets start from the 2010-2011 season and stretch to the 2021-2022 season. All of the major data gathering and cleaning occurs in the next three code blocks.


The following block does all of the prep work before we start to read the data. It first establishes where the data is stored in the repository so that it can be read by the program. It then creates a list of labels that will be used by the rotowire data (the rotowire dataset is formatted differently than the moneypuck dataset, so we need to do this extra step before proceeding). It then establishes a points breakdown for each relevant stat; this is used later on to calculate fantasy points for each player.

In [2]:
#The current working directory is the main repository directory; these lines set the path to where the data is
path = os.getcwd()
data_path = path + '\\data'

#This array makes it easier to format the rotowire data
rw_labels = ["name", "Team", "Pos", "Games", "Goals", "Assists", "Pts", "+/-", "PIM", "SOG", "GWG", "PP_Goals", "PP_Assists", "SH_Goals", "SH_Assists", "Hits", "Blocked_Shots"]

#This is the breakdown of how many fantasy points a player gets for each category
points_dict = {"Goals":5, "Assists":3, "+/-":1.5, "PIM":-0.25, "PP_Goals":4, "PP_Assists":2, "SH_Goals":6, "SH_Assists":4, "Faceoffs_Won":0.25, "Faceoffs_Lost":-0.15, "Hits":0.5, "Blocked_Shots":0.75 }


The following block takes all of the data in the repository and turns it into year-by-year player data. For each year from 2010 to 2021, the for-loop reads the rotowire and moneypuck data from the csv files in the repository, merges the datasets together, calculates the player's fantasy points for that season, does some formatting, then adds it to the "yearly_player_data" list. This list can be used later on for turning into ML-readable data.

In [3]:
#I have data from the 2010-2011 season through the 2021-2022 season.
#By the end of this block, there will be 12 seasons-worth of data in the "data" variable
yearly_player_data = []

for i in range(2010, 2022):
    new_data = []
    
    #Imports the rotowire and moneypuck datasets from the selected year into rdf and mdf
    rdf = pd.read_csv(data_path + '\\rotowire_data\\rotowire{}.csv'.format(str(i)))
    mdf = pd.read_csv(data_path + '\\moneypuck_data\\moneypuck{}.csv'.format(str(i)))
    
    #Formats the rotowire data
    rdf.set_axis(rw_labels, axis=1, inplace=True)
    rdf.drop(index=rdf.index[0], axis=0, inplace=True)
    
    #The Moneypuck data has information about 5-on-5, 5-on-4, 4-on-5, other, and all.
    #For this project I'm just focused on "all" since I suspect it'll give me the best results.
    mdf = mdf[mdf["situation"] == "all"]
    
    #Combines the "Name" and "Team" columns (There are some players with the same name on different teams)
    rdf["name"] = rdf["name"] + "-" + rdf["Team"]
    mdf["name"] = mdf["name"] + "-" + mdf["team"]
    
    
    
    #Merges the rotowire and moneypuck dataframes
    new_data = pd.merge(rdf, mdf, on="name")
    
    #Changes the name of a few columns in the new dataframe
    new_data = new_data.rename(columns={"name":"Name","faceoffsWon":"Faceoffs_Won","faceoffsLost":"Faceoffs_Lost"})
    
    #This section calculates each player's total fantasy output for that year
    cols = new_data.columns
    fant_points = [0 for i in range(len(new_data))]
    for i in range(len(new_data)):
        for j in range(len(new_data.iloc[i])):
            mult = points_dict.get(cols[j], 0)
            if mult != 0:
                fant_points[i] += mult*int(new_data.iloc[i, j])
    
    #Adds the players' fantasy points to the new_data dataframe
    new_data["Fantasy_Points"] = fant_points
    
    new_data = new_data.drop_duplicates()
    
    #Adds new_data to the "data" array
    yearly_player_data.append(new_data)
                
    



The following block takes the yearly data and turns it into ML-readable data. For this project, I am creating different models that use data from the past one year, the past two years, and the past three years, and seeing how much they differ in terms of efficacy.

In [4]:
ml_data_one_year = pd.DataFrame()
ml_data_two_year = pd.DataFrame()
ml_data_three_year = pd.DataFrame()
for i in range(2011, 2022):
    arr = [yearly_player_data[i-2011]]
    points_df = yearly_player_data[i-2010]
    temp = mx.merge_dataframes(arr, points_df)
    ml_data_one_year = pd.concat([ml_data_one_year, temp], ignore_index=True)
    
for i in range(2012, 2022):
    arr = [yearly_player_data[i-2012], yearly_player_data[i-2011]]
    points_df = yearly_player_data[i-2010]
    temp = mx.merge_dataframes(arr, points_df)
    ml_data_two_year = pd.concat([ml_data_two_year, temp], ignore_index=True)
    
for i in range(2013, 2022):
    arr = [yearly_player_data[i-2013], yearly_player_data[i-2012], yearly_player_data[i-2011]]
    points_df = yearly_player_data[i-2010]
    temp = mx.merge_dataframes(arr, points_df)
    ml_data_three_year = pd.concat([ml_data_three_year, temp], ignore_index=True)



Section 3:

Now that we have the data that ml models can read, we can now train the models. For this project, I'm using multi-layer perceptrons (MLPRegressor) and Random Forests (RandomForestRegressor). I'm making six total models: a MLP each for the one- two- and three-year data, and a Random Forest each for the one- two- and three-year data.

ONE YEAR:

In [5]:
arr = mx.separate_fantasy_points(ml_data_one_year)
X = mx.reformat_df(arr[0])
y = arr[1]

X_train_one, X_test_one, y_train_one, y_test_one = train_test_split(X, y)

one_year_regr = MLPRegressor(max_iter=1000)
one_year_regr.fit(X_train_one, y_train_one)

one_year_RF = RandomForestRegressor()
one_year_RF.fit(X_train_one, y_train_one)



RandomForestRegressor()

TWO YEAR:

In [6]:
arr = mx.separate_fantasy_points(ml_data_two_year)
X = mx.reformat_df(arr[0])
y = arr[1]

X_train_two, X_test_two, y_train_two, y_test_two = train_test_split(X, y)

two_year_regr = MLPRegressor(max_iter=1000)
two_year_regr.fit(X_train_two, y_train_two)

two_year_RF = RandomForestRegressor()
two_year_RF.fit(X_train_two, y_train_two)



RandomForestRegressor()

THREE YEAR:

In [7]:
arr = mx.separate_fantasy_points(ml_data_three_year)
X = mx.reformat_df(arr[0])
y = arr[1]

X_train_three, X_test_three, y_train_three, y_test_three = train_test_split(X, y)

three_year_regr = MLPRegressor(max_iter=1000)
three_year_regr.fit(X_train_three, y_train_three)

three_year_RF = RandomForestRegressor()
three_year_RF.fit(X_train_three, y_train_three)



RandomForestRegressor()

Section 4: Analysis

Now that we have the models trained, we can analyze them. We'll be analyzing the data in two ways: first, we'll see how accurate the actual points predictions are using the mean_absolute_error.

In [8]:
y_pred = one_year_regr.predict(X_test_one)
print(mean_absolute_error(y_test_one, y_pred))

y_pred = two_year_regr.predict(X_test_two)
print(mean_absolute_error(y_test_two, y_pred))

y_pred = three_year_regr.predict(X_test_three)
print(mean_absolute_error(y_test_three, y_pred))

y_pred = one_year_RF.predict(X_test_one)
print(mean_absolute_error(y_test_one, y_pred))

y_pred = two_year_RF.predict(X_test_two)
print(mean_absolute_error(y_test_two, y_pred))

y_pred = three_year_RF.predict(X_test_three)
print(mean_absolute_error(y_test_three, y_pred))

84.61103284864018
81.21376790046595
85.9991563495849
72.69802166064981
78.7745206185567
81.48080575539568


The MAE for all of the models range between around 70 and around 95. Given that most players in the league finish with a fantasy point total in the hundreds, we can see that the predicted points values aren't very accurate to the real-life values. However, we are less concerned with the actual points total that a player will have, and more concerned with their rank within the rest of the league. To look at this, we will rank the players both in terms of predicted fantasy points for a season, and actual fantasy points for a season. (I'll incorporate this at a later time)

Section 5: Predictions for Next Year

Now that we've taken a look at the accuracy of the model, we'll see what the models think will happen in the 2022-2023 season. 

In [9]:
one_year_pred = yearly_player_data[11].copy()
one_year_pred.drop(columns=["Fantasy_Points"], inplace=True)
one_year_df = mx.get_name_predictions(one_year_regr, one_year_pred)
one_year_df.sort_values(by="Prediction", ascending=False, inplace=True)
display(one_year_df)


,Name,Prediction
48,Auston Matthews-TOR,334.514477
25,Patrice Bergeron-BOS,308.007057
394,Josh Norris-OTT,304.958204
0,Elias Lindholm-CGY,294.141818
37,Nathan MacKinnon-COL,289.374012
...,...,...
238,Walker Duehr-CGY,25.534216
252,David Gustafsson-WPG,22.713968
289,Pavel Dorofeyev-VGK,22.482483
281,Guillaume Brisebois-VAN,19.674072


In [10]:
two_year_pred = [yearly_player_data[i] for i in [10,11]]
two_year_pred = mx.merge_dataframes(two_year_pred, yearly_player_data[11])
two_year_pred.drop(columns=["Predicted_Fantasy_Points"], inplace=True)
two_year_df = mx.get_name_predictions(two_year_regr, two_year_pred)
two_year_df.sort_values(by="Prediction", ascending=False, inplace=True)
display(two_year_df)

,Name,Prediction
0,Auston Matthews-TOR,464.250649
7,Aleksander Barkov-FLA,426.576166
3,Mikko Rantanen-COL,421.562903
21,Nathan MacKinnon-COL,412.567376
15,Patrice Bergeron-BOS,384.301223
...,...,...
227,Victor Soderstrom-ARI,7.361987
268,Matt Kiersted-FLA,5.817555
284,Martin Kaut-COL,4.224930
185,Ty Dellandrea-DAL,-8.940074


In [11]:
three_year_pred = [yearly_player_data[i] for i in [9,10,11]]
three_year_pred = mx.merge_dataframes(three_year_pred, yearly_player_data[11])
three_year_pred.drop(columns=["Predicted_Fantasy_Points"], inplace=True)
three_year_df = mx.get_name_predictions(three_year_regr, three_year_pred)
three_year_df.sort_values(by="Prediction", ascending=False, inplace=True)
display(three_year_df)

,Name,Prediction
6,Nathan MacKinnon-COL,390.200394
1,Auston Matthews-TOR,372.769671
41,Aleksander Barkov-FLA,347.385502
10,Patrice Bergeron-BOS,343.884658
48,Mikko Rantanen-COL,340.259162
...,...,...
197,Libor Hajek-NYR,85.104241
171,Taro Hirose-DET,84.281374
199,Tim Gettinger-NYR,63.020883
169,Martin Kaut-COL,52.573115


In [12]:
one_year_pred = yearly_player_data[11].copy()
one_year_pred.drop(columns=["Fantasy_Points"], inplace=True)
one_year_df = mx.get_name_predictions(one_year_RF, one_year_pred)
one_year_df.sort_values(by="Prediction", ascending=False, inplace=True)
display(one_year_df)

,Name,Prediction
53,Sidney Crosby-PIT,403.2050
25,Patrice Bergeron-BOS,391.4950
37,Nathan MacKinnon-COL,389.4675
7,Aleksander Barkov-FLA,383.4290
20,Gabriel Landeskog-COL,380.9080
...,...,...
290,Rhett Gardner-DAL,34.7435
288,Adam Beckman-MIN,31.6855
186,Fredrik Karlstrom-DAL,30.5730
187,Martin Kaut-COL,29.3835


In [13]:
two_year_pred = [yearly_player_data[i] for i in [10,11]]
two_year_pred = mx.merge_dataframes(two_year_pred, yearly_player_data[11])
two_year_pred.drop(columns=["Predicted_Fantasy_Points"], inplace=True)
two_year_df = mx.get_name_predictions(two_year_RF, two_year_pred)
two_year_df.sort_values(by="Prediction", ascending=False, inplace=True)
display(two_year_df)

,Name,Prediction
15,Patrice Bergeron-BOS,417.0300
9,Sidney Crosby-PIT,414.6505
22,Gabriel Landeskog-COL,413.4640
4,Brad Marchand-BOS,409.2750
21,Nathan MacKinnon-COL,407.1415
...,...,...
264,Zac Jones-NYR,52.5885
292,Reese Johnson-CHI,48.2595
272,Tanner Laczynski-PHI,43.5080
270,Tim Gettinger-NYR,38.2990


In [14]:
three_year_pred = [yearly_player_data[i] for i in [9,10,11]]
three_year_pred = mx.merge_dataframes(three_year_pred, yearly_player_data[11])
three_year_pred.drop(columns=["Predicted_Fantasy_Points"], inplace=True)
three_year_df = mx.get_name_predictions(three_year_RF, three_year_pred)
three_year_df.sort_values(by="Prediction", ascending=False, inplace=True)
display(three_year_df)

,Name,Prediction
6,Nathan MacKinnon-COL,394.4890
14,Brad Marchand-BOS,386.3735
41,Aleksander Barkov-FLA,384.3030
1,Auston Matthews-TOR,375.9725
0,David Pastrnak-BOS,370.7040
...,...,...
179,Ville Heinola-WPG,45.8180
169,Martin Kaut-COL,43.8630
178,David Gustafsson-WPG,38.3540
199,Tim Gettinger-NYR,33.3545


Section 6: Conclusion and Next Steps

The predictions made by all of these models make sense; all of the predicted top players are still some of the top players in the league this year, and many of the predictions match up with predictions made by ESPN. One step that could be done is aggregating the six models together to get an average points prediction, and listing the players that way. Another thing that can be done to improve the models is incorporate injury data; there are some elite players that were injured in some part of the past three years, and their predictions are more pessemistic than other players. Another improvement could be to try and scale for the COVID-shortened 2019-2020 and 2020-2021 seasons. There were many logistical issues that contributed to fewer games and lower scoring in those years, and a scaling of goal/assist values could be beneficial to the models.

Overall, I'm happy with how the models performed. After the 2022-2023 regular season, I will see how well they were able to predict some of the outliers, and I'll use that new data to make a prediction for the 2023-2024 season.

ACKNOWLEDGEMENTS:

Thank you to Rotowire and Moneypuck for making your NHL data easy for someone like me to utilize in a project like this, and thank you to Peter Tanner of Moneypuck not just for creating such a valuable resource, but for being responsive to questions I was having about your dataset.